# a study of pandas dataframes with different levels

the goal of this work is to study how index and column names are presented in html `table`s.

start with a dataframe with multi indexes. 
each level of the study removes indexes rather than add.

In [ ]:
    import pandas

    df = pandas.DataFrame(
        index=pandas.MultiIndex.from_product(
            [[100], [10, 20, 30], [1, 2]]
        ),
        columns=pandas.MultiIndex.from_product(
            [["AZ"], ["A", "Z"], ["A", "B", "C"]]
        )
    ).rename_axis(
        index=["hundreds", "tens", "ones"],
        columns=["outer", "middle", "inner"]
    ).head(2).fillna("")

iterate through different pairings of row and column names

In [52]:
    for i in range(1, df.index.nlevels):
        for j in range(i, df.index.nlevels):
            data = df
            while data.index.nlevels > i: 
                data = data.droplevel(0, axis=0)
                if data.index.nlevels == 1: break
            while data.columns.nlevels > j:
                data = data.droplevel(0, axis=1)
                if data.columns.nlevels == 1:
                    break
            for row_names in ([None]*data.index.nlevels, data.index.names):
                for col_names in ([None]*data.columns.nlevels, data.columns.names):
                    title = str(len(row_names)) + [" un", " "][any(row_names)] + F"named rows and {len(col_names)} " + ["un", ""][any(col_names)] + "named columns"
                    out = data.rename_axis(index=row_names, columns=col_names).style.set_caption(title)
                    display(out)
            display({"text/html": "<hr/>"}, raw=True)

,A,B,C,A,B,C
1,,,,,,
2,,,,,,


inner,A,B,C,A,B,C
1,,,,,,
2,,,,,,


,A,B,C,A,B,C
ones,,,,,,
1,,,,,,
2,,,,,,


inner,A,B,C,A,B,C
ones,,,,,,
1,,,,,,
2,,,,,,
